# Budget Summary Extractor – 2026
This notebook extracts the **2026 Budget** (Column H) for the following key labels found in **Column B**:
- `All OpEx IT Expenses`
- `Total Capital Expense`
- `Total Recommended Savings`
The results are displayed per worksheet (site) and formatted as rounded dollar values.

In [5]:
import pandas as pd
import os
from pathlib import Path

# Configuration
excel_filename = "2026 BRM Budget Template - 7.07.2025_Offline.xlsx"

# Try different possible locations for the Excel file (prioritizing known location)
possible_paths = [
    f"../data/raw/{excel_filename}",  # Known correct location
    f"data/raw/{excel_filename}",  # Alternative path
    excel_filename,  # Current directory
    f"C:/Users/cglynn/Documents/{excel_filename}",  # Documents
    f"C:/Users/cglynn/Downloads/{excel_filename}"   # Downloads
]

excel_path = None
for path in possible_paths:
    if os.path.exists(path):
        excel_path = path
        print(f"✅ Found Excel file at: {path}")
        break

if excel_path is None:
    print(f"❌ Excel file '{excel_filename}' not found in any of these locations:")
    for path in possible_paths:
        print(f"   - {os.path.abspath(path)}")
    print("\nPlease place the file in one of the above locations or update the excel_filename variable.")
    raise FileNotFoundError(f"Excel file not found: {excel_filename}")

# Load the Excel file
try:
    xls = pd.ExcelFile(excel_path)
    print(f"📊 Successfully loaded Excel file with {len(xls.sheet_names)} sheets")
except Exception as e:
    print(f"❌ Error loading Excel file: {e}")
    raise

# List of target worksheets to analyse
target_sheets = [
    "224 - Gillingham", "227 - Stonehouse", "230 - Isai", "231 - Blois & Cergy",
    "234 - Izmir Plant", "235 - Beval", "247 - Rzeszow", "249 - Krakow",
    "576 - Dusseldorf", "126 - Warwick", "128 - Buckinghamshire", "149 - Izmir"
]

print(f"🎯 Target sheets to analyze: {len(target_sheets)}")

# Check which target sheets exist in the workbook
available_target_sheets = [sheet for sheet in target_sheets if sheet in xls.sheet_names]
missing_sheets = [sheet for sheet in target_sheets if sheet not in xls.sheet_names]

print(f"✅ Available target sheets: {len(available_target_sheets)}")
if missing_sheets:
    print(f"⚠️  Missing sheets: {missing_sheets}")

# Target labels to extract
target_labels = [
    "All OpEx IT Expenses", 
    "Total Capital Expense", 
    "Total Recommended Savings"
]

print(f"🏷️  Target labels: {target_labels}")

✅ Found Excel file at: ../data/raw/2026 BRM Budget Template - 7.07.2025_Offline.xlsx
📊 Successfully loaded Excel file with 46 sheets
🎯 Target sheets to analyze: 12
✅ Available target sheets: 11
⚠️  Missing sheets: ['576 - Dusseldorf']
🏷️  Target labels: ['All OpEx IT Expenses', 'Total Capital Expense', 'Total Recommended Savings']


In [6]:
# Extract rows where Column B matches any of the target row labels
budget_records = []
processing_log = []

print("🔍 Processing sheets for budget data extraction...\n")

for sheet in available_target_sheets:
    print(f"Processing: {sheet}")
    try:
        df = xls.parse(sheet)
        df.columns = df.columns.map(str)
        
        # Check if we have enough columns
        if df.shape[1] <= 7:
            processing_log.append(f"❌ {sheet}: Not enough columns (only {df.shape[1]})")
            continue
        
        sheet_found_labels = []
        
        for label in target_labels:
            matches = df[df.iloc[:, 1].astype(str).str.strip() == label]
            
            if matches.empty:
                processing_log.append(f"⚠️  {sheet} - '{label}': Not found")
                continue
                
            if len(matches) > 1:
                processing_log.append(f"⚠️  {sheet} - '{label}': Multiple matches found, using first one")
            
            # Extract the budget value from column H (index 7)
            budget_val = pd.to_numeric(matches.iloc[0, 7], errors='coerce')
            
            if pd.isna(budget_val):
                processing_log.append(f"⚠️  {sheet} - '{label}': Invalid/missing budget value")
                budget_val = 0
            
            budget_records.append({
                "Site": sheet,
                "Row Label": label,
                "2026 Budget ($USD)": budget_val,
                "Raw Value": matches.iloc[0, 7]  # Keep original for debugging
            })
            
            sheet_found_labels.append(label)
            processing_log.append(f"✅ {sheet} - '{label}': ${budget_val:,.0f}")
        
        if sheet_found_labels:
            print(f"   Found {len(sheet_found_labels)}/{len(target_labels)} target labels")
        else:
            print(f"   ❌ No target labels found")
            
    except Exception as e:
        processing_log.append(f"❌ {sheet}: Error processing - {str(e)}")
        print(f"   ❌ Error: {str(e)}")

# Display processing summary
print(f"\n📋 Processing Summary ({len(processing_log)} entries):")
for log_entry in processing_log:
    print(log_entry)

# Create and format results
if budget_records:
    summary_df = pd.DataFrame(budget_records)
    
    # Calculate totals by label
    print(f"\n💰 Budget Totals by Category:")
    print("=" * 40)
    
    totals_by_label = summary_df.groupby('Row Label')['2026 Budget ($USD)'].sum()
    for label, total in totals_by_label.items():
        print(f"{label:<25} ${total:>15,.0f}")
    
    grand_total = summary_df['2026 Budget ($USD)'].sum()
    print(f"{'GRAND TOTAL':<25} ${grand_total:>15,.0f}")
    
    # Format the budget column for display
    display_df = summary_df.copy()
    display_df["2026 Budget ($USD)"] = (
        display_df["2026 Budget ($USD)"]
        .apply(lambda x: f"${int(round(x)):,}" if pd.notna(x) else "$0")
    )
    
    # Remove the raw value column for display
    display_df = display_df.drop(columns=['Raw Value'])
    
    print(f"\n📊 Successfully extracted {len(budget_records)} budget entries from {len(available_target_sheets)} sheets")
    
    # Display results sorted by site and label
    display_df_sorted = display_df.sort_values(['Site', 'Row Label'])
    display_df_sorted
else:
    print("\n❌ No budget data extracted. Please check the file structure and sheet names.")
    pd.DataFrame()  # Return empty dataframe

🔍 Processing sheets for budget data extraction...

Processing: 224 - Gillingham
   Found 3/3 target labels
Processing: 227 - Stonehouse
   Found 3/3 target labels
Processing: 230 - Isai
   Found 3/3 target labels
Processing: 231 - Blois & Cergy
   Found 3/3 target labels
Processing: 234 - Izmir Plant
   Found 3/3 target labels
Processing: 235 - Beval
   Found 3/3 target labels
Processing: 247 - Rzeszow
   Found 3/3 target labels
Processing: 249 - Krakow
   Found 3/3 target labels
Processing: 126 - Warwick
   Found 3/3 target labels
Processing: 128 - Buckinghamshire
   Found 3/3 target labels
Processing: 149 - Izmir
   Found 3/3 target labels

📋 Processing Summary (33 entries):
✅ 224 - Gillingham - 'All OpEx IT Expenses': $0
✅ 224 - Gillingham - 'Total Capital Expense': $0
✅ 224 - Gillingham - 'Total Recommended Savings': $0
✅ 227 - Stonehouse - 'All OpEx IT Expenses': $0
✅ 227 - Stonehouse - 'Total Capital Expense': $0
✅ 227 - Stonehouse - 'Total Recommended Savings': $0
✅ 230 - Isai -

In [7]:
# 📈 Detailed Analysis and Export
if budget_records:
    print("🔍 Detailed Budget Analysis:")
    print("=" * 50)
    
    # Analysis by site
    site_totals = summary_df.groupby('Site')['2026 Budget ($USD)'].sum().sort_values(ascending=False)
    print(f"\n🏢 Total Budget by Site (Top 5):")
    for site, total in site_totals.head().items():
        print(f"   {site:<25} ${total:>12,.0f}")
    
    # Sites with zero budgets across all categories
    zero_budget_sites = site_totals[site_totals == 0].index.tolist()
    if zero_budget_sites:
        print(f"\n⚠️  Sites with zero budgets across all categories: {len(zero_budget_sites)}")
        for site in zero_budget_sites:
            print(f"   - {site}")
    
    # Analysis by category
    print(f"\n📊 Budget Distribution by Category:")
    for label in target_labels:
        label_data = summary_df[summary_df['Row Label'] == label]
        if not label_data.empty:
            label_total = label_data['2026 Budget ($USD)'].sum()
            sites_with_data = len(label_data[label_data['2026 Budget ($USD)'] > 0])
            print(f"   {label:<25} ${label_total:>12,.0f} ({sites_with_data} sites)")
        else:
            print(f"   {label:<25} ${'0':>12} (0 sites)")
    
    # Create pivot table for better visualization
    print(f"\n📋 Budget Matrix (Sites × Categories):")
    pivot_df = summary_df.pivot(index='Site', columns='Row Label', values='2026 Budget ($USD)')
    pivot_df = pivot_df.fillna(0)
    
    # Add totals
    pivot_df['TOTAL'] = pivot_df.sum(axis=1)
    
    # Format for display
    pivot_display = pivot_df.copy()
    for col in pivot_display.columns:
        pivot_display[col] = pivot_display[col].apply(lambda x: f"${x:,.0f}")
    
    print(pivot_display.to_string())
    
    # Export to CSV
    output_path = "../data/processed/extended_budget_2026_summary.csv"
    try:
        # Ensure directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Export detailed data
        export_df = summary_df.drop(columns=['Raw Value']).copy()
        export_df.to_csv(output_path, index=False)
        
        # Export pivot table
        pivot_export_path = "../data/processed/budget_matrix_2026.csv"
        pivot_df.to_csv(pivot_export_path)
        
        print(f"\n💾 Detailed results exported to: {os.path.abspath(output_path)}")
        print(f"💾 Budget matrix exported to: {os.path.abspath(pivot_export_path)}")
        
    except Exception as e:
        print(f"❌ Export failed: {e}")
        
else:
    print("No data available for analysis.")

🔍 Detailed Budget Analysis:

🏢 Total Budget by Site (Top 5):
   126 - Warwick             $   4,638,827
   128 - Buckinghamshire     $           0
   149 - Izmir               $           0
   224 - Gillingham          $           0
   227 - Stonehouse          $           0

⚠️  Sites with zero budgets across all categories: 10
   - 128 - Buckinghamshire
   - 149 - Izmir
   - 224 - Gillingham
   - 227 - Stonehouse
   - 230 - Isai
   - 231 - Blois & Cergy
   - 234 - Izmir Plant
   - 235 - Beval
   - 247 - Rzeszow
   - 249 - Krakow

📊 Budget Distribution by Category:
   All OpEx IT Expenses      $   4,633,077 (1 sites)
   Total Capital Expense     $       5,750 (1 sites)
   Total Recommended Savings $           0 (0 sites)

📋 Budget Matrix (Sites × Categories):
Row Label             All OpEx IT Expenses Total Capital Expense Total Recommended Savings       TOTAL
Site                                                                                                  
126 - Warwick          

In [8]:
# 🔧 Troubleshooting: Explore Excel File Structure
# Uncomment and run this section if you need to investigate the file structure

# print("📋 All available sheets in the workbook:")
# for i, sheet_name in enumerate(xls.sheet_names, 1):
#     print(f"   {i:2d}. {sheet_name}")

# # Example: Look at the first few rows of a specific sheet to understand structure
# sample_sheet = available_target_sheets[0] if available_target_sheets else xls.sheet_names[0]
# print(f"\n🔍 Sample data from '{sample_sheet}' (first 15 rows, columns A-J):")
# sample_df = xls.parse(sample_sheet)
# print(sample_df.iloc[:15, :10].to_string())

# # Look for all potential budget-related rows
# print(f"\n🔍 All rows containing budget-related keywords in column B of '{sample_sheet}':")
# keywords = ['OpEx', 'Capital', 'Savings', 'Budget', 'Total', 'Expense']
# for keyword in keywords:
#     matching_rows = sample_df[sample_df.iloc[:, 1].astype(str).str.contains(keyword, case=False, na=False)]
#     if not matching_rows.empty:
#         print(f"\n   Rows containing '{keyword}':")
#         for idx, row in matching_rows.iterrows():
#             print(f"      Row {idx}: {row.iloc[1]}")

print("💡 Uncomment the code above to explore the Excel file structure if needed.")

💡 Uncomment the code above to explore the Excel file structure if needed.
